<div style="text-align: right"> &uarr;   Ensure Kernel is set to  &uarr;  </div><br><div style="text-align: right"> 
conda_amazonei_pytorch_latest_p36  </div>

# Custom FastAI Model Inference using PyTorch Base Container

This notebook walks you through setting up FastAI Model you just trained using BYOC for inference using endpoint. First we need to create a Predictor class to accept jpeg images as input and output JSON. The default behaviour is to accept a numpy array.

Update Sagemaker SDK if not already done so

In [ ]:
!pip install -U sagemaker

Import necessary libraries

In [ ]:
import sagemaker
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()

Please replace the **your-trained-model-uri** with the S3 URI location of your Fast AI Model from the previous notebook

In [ ]:
s3_model_loc='your-trained-model-uri'

## Update the Image Predictor Class

In the next cell, we will update the predictor class to accept json serializer and deserializer and accept application/x-image content type

In [ ]:
from sagemaker import Predictor
jpeg_serializer = sagemaker.serializers.IdentitySerializer("application/x-image")
json_deserializer = sagemaker.deserializers.JSONDeserializer()


class ImagePredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(ImagePredictor, self).__init__(
            endpoint_name,
            sagemaker_session=sagemaker_session,
            serializer=jpeg_serializer,
            deserializer=json_deserializer,
        )

### Ineference Script
Create an Inference Script along with any libraries we need installed inside requirements.txt and save them in the inf_src folder

In [ ]:
%cat inf_src/serve.py

In [ ]:
%cat inf_src/requirements.txt

### Prepare the model
Using the model and ImagePredictor class from above, prepare the model for deployment as an endpoint and provide a serving script that can upack the model

In [ ]:
from sagemaker.pytorch import PyTorchModel
from sagemaker.utils import name_from_base

model=PyTorchModel(model_data=s3_model_loc, 
                   name=name_from_base("fastai-custom-cont-mod"),
                   role=role, 
                   framework_version='1.8.0',
                   py_version='py3',
                   entry_point='inf_src/serve.py',
                   source_dir= 'inf_src',
                   predictor_cls=ImagePredictor)

### Deploy the model 

Deploy the model to the end point using ml.m4.xlarge

In [ ]:
%%time 
from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
print(f'Your endpoint name is \n{predictor.endpoint_name}')

## Create a Predictor to make inference 

**NOTE** Replace **your-endpoint-name** with your endpoint name in the cell below

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.predictor import json_serializer

#copy the endpoint name and update from sagemaker console --> inference-endpoints
endpoint_name='your-endpoint-name'

predictor=Predictor(endpoint_name=endpoint_name, 
                    sagemaker_session=sagemaker_session,serializer=jpeg_serializer)

with open('../data/test/Roundabout/R1.png', 'rb') as f:
    img_byte=f.read()
    print(predictor.predict(img_byte))

### Using Boto3 Library

In [ ]:
import boto3
client=boto3.client('sagemaker-runtime')
im_name="../data/test/Roundabout/R2.png"

response = client.invoke_endpoint(
EndpointName=endpoint_name,
ContentType='application/x-image',
Body=open(im_name, 'rb').read())

In [ ]:
import json
json.loads(response['Body'].read().decode("utf-8"))